# Mengaplikasikan Perceptron Dengan Scikit-learn Library

> Perbincangan berkenaan penggunaan perceptron menggunakan sklearn library dan python

- toc: false 
- badges: true
- comments: true
- categories: [deep learning]
- image: images/
- author: Megat Norulazmi

Assalamualaikum dan selamat berjumpa kembali.

Seperti yang telah saya nukilkan pada [artikel yang lepas](https://megatazm.github.io/MyDLBlog/deep%20learning/2020/12/29/basic-dl.html), saya telah menerangkan dengan serba ringkas berkenaan mekanisma perceptron dan menunjukkan contoh penggunaan mudah perceptron menggunakan sklearn library (ataupun scikit-learn). Di artikel ini, saya akan menulis penggunaan perceptron dengan aturan proses machine learning yang lebih lengkap daripada sebelum ini. 

Untuk kali ini, saya masih menggunakan sklearn library. Mungkin ada yang berkata-kata, 
>"Asyik sebut-sebut keras, tensorflow dan pytorch jer sebelum ini, tapi tak pakai-pakai pun". 

InsyaAllah saya cuba perkenalkan asas penggunaanya di artikel yang seterusnya ya.

Berikut adalah pecahan proses yang kita akan lakukan pada kali ini:-

1.   Import library sklearn yang diperlukan
2.   Menghasilkan data buatan menggunakan library sklearn **make_classification**
3.   Bina perceptron menggunakan library sklearn **Perceptron**
4.   Melakukan model validation dengan stratified k-fold cross validation menggunakan library sklearn **RepeatedStratifiedKFold**
5.   Train model perceptron menggunakan data buatan tersebut 
6. Melakukan hyperparameter tuning untuk learning rate dan kekerapan iteration melalui sklearn library **GridSearchCV**
7. Memaparkan prestasi validation model tersebut berdasarkan pada kombinasi beberapa nilai learning rate dan iteration yang berbeza.
8. Menguji prestasi model menggunakan test data dengan kombinasi learning rate dan kekerapan iteration pilihan yang terbaik, kemudian memaparkan prestasi ketepatannya.



Pertama sekali, import library yang bakal kita gunakan,

*   make_classification - Fungsi untuk menghasilkan data buatan
*   GridSearchCV - Fungsi untuk melakukan hyperparameter tuning
*   RepeatedStratifiedKfold - Fungsi untuk menjalankan stratified k-fold cross validation yang boleh ditentukan kekerapan ulangan.
*  Perceptron - Fungsi untuk membina architecture algoritma perceptron

In [20]:
# import sklearn library yang akan digunakan
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import Perceptron

Tujuan kita menggunakan fungsi make_classification ini adalah menghasilkan data yang sesuai untuk perceptron melakukan klasifikasi binari. Ini kerana perceptron hanya mampu melakukan klasifikasi pada data yang "***linearly separable***", iaitu data yang mudah diasingkan kepada dua kelas dengan menggunakan satu garisan linear. Fungsi make_classification ini juga boleh digunakan untuk menghasilkan pelbagai jenis data klasifikasi yang kompleks. Ia sangat berguna untuk menguji prestasi dan melakukan perbandingan di antara pelbagai jenis algoritma klasifikasi yang lama dan baharu. Penerangan berkenaan penggunaan fungsi ini boleh di rujuk di [sini](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html). Bagi tujuan contoh ini, kita hanya akan menggunakan parameternya yang berikut,

*   **n_samples** - Untuk menentukan jumlah instance atau rekod. Di sini saya setkan dengan nilai n_samples=1000.
*   **n_feature** - Untuk menentukan jumlah input fitur. Di sini saya setkan n_feature=10
*   **n_informative** - Untuk menentukan samada input fitur yang telah ditetapkan jumlahnya sebentar tadi adalah berinformatif. Nilai tetapannya mestilah tidak melebihi dengan nilai n_feature. Di sini saya setkan nilai n_informative = 10
* **n_redundant** - Oleh kerana kita telah menetapkan kesemua 10 feature kita adalah informatif maka di sini n_redundant = 0. Jika contohnya kita setkan n_redundant = 5, maka mestilah nilai n_informative=5 kerana total fitur adalah 10 (jika n_informative=4, n_redundant = 5, maka fitur selebihnya secara default disetkan sebagai noise). Lagi tinggi fitur informatif maka lagi mudahlah model perceptron itu di latih dan memperolehi ketepatan ramalan yang tinggi.
* **random_state** - Jujukan data instance atau rekod yang akan dihasilkan nanti adalah random. Namun, untuk memastikan data random ini adalah "*reproducible*", maksudnya untuk memastikan data ini sentiasa sama jika kita menghasilkannya berulang kali. Oleh itu, anda perlu setkan nilai integer yang sama pada random_state itu. Seperti contoh, di sini saya setkan random_state = 1.



In [21]:
# Menghasilkan dataset buatan. Cuba awak run kod ini berkali-kali. Hasilnya 
# tetap sama kan? Cuba tukar nilai random_state=2, kemudian run. Sama juga ker?
X, y = make_classification(n_samples=1000, n_features=10, n_informative=10, n_redundant=0, random_state=1)

#uncomment ini untuk lihat data yang terhasil
#print (X, y)

Untuk membina architecture NN perceptron, kita menggunakan fungsi perceptron dari library sklearn (tak perlu bina "*from scratch*"). Fungsi ini juga ada banyak setting parameternya. Anda boleh rujuk di [sini](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Perceptron.html?highlight=perceptron#sklearn.linear_model.Perceptron). Tapi, untuk contoh ini kita gunakan setting default sahaja. Setting parameter learning rate dan iteration pada fungsi perceptron ini akan kita lakukan melalui fungsi GridSearchCV. Hasilnya di simpan di dalam variable ***model***.

In [22]:
# Bina NN default perceptron
model = Perceptron()

Sebelum menjalankan proses training dengan stratified k-fold cross validation (Saya ada berikan penerangan berkenaan proses ini di [artikel](https://megatazm.github.io/MyDLBlog/deep%20learning/2020/12/23/AI-ML-DL.html) yang pertama), kita perlu tetapkan nilai parameter pada fungsi RepeatedStratifiedKFold terlebih dahulu. Fungsi ini hanya ada 3 parameter sahaja yang perlu ditentukan nilainya. Penerangan lengkapnya ada di [sini](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RepeatedStratifiedKFold.html?highlight=repeatedstratifiedkfold#sklearn.model_selection.RepeatedStratifiedKFold). Setting parameternya adalah seperti berikut,

*   n_splits - Adalah untuk menentukan nilai k. Dalam contoh ini, saya setkan k=10 (n_splits=10), bermaksud data ini akan diasingkan sebanyak 10 bahagian (9 data training dan 1 data testing)
*   n_repeats - Menetapkan kekerapan proses 10-fold cross validation yang ingin di ulangi. Saya setkan n_repeats=3.
*  random_state - Saya rasa anda boleh agak tujuan parameter ini kan? Bagus! anda memang bijak.

Hasil tetapan ini kemudiannya di simpan di variable ***cv***.

In [23]:
# tetapkan setting model validation stratified k-fold cross validation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

Di sini kita tetapkan konfigurasi hyperparameter tuning yang ingin dijalankan ke dalam variable ***grid***. Di contoh ini, saya hanya tetapkan parameter '**max_iter**' untuk iteration dan '**eta0**' untuk learning rate sahaja, setiap satunya dengan 5 nilai-nilai yang berbeza (parameter 'max_iter dan 'eta0' ini adalah berdasarkan pada tetapan parameter yang di sokong oleh fungsi perceptron( )). Akhir sekali, di fungsi GridSearchCV, masukkan variable **model**, **grid** dan **cv** yang telah kita konfigurasikan tadi ke tempat yang betul dengan tetapan parameter yang lain seperti **scoring='accuracy'** untuk memaparkan ketepatan prestasinya dan parameter **n_jobs=-1** supaya pemprosesan ini menggunakan kesemua cpu yang ada di komputer. Hasil tetapan fungsi GridSearchCV ini kemudiannya di simpan ke dalam variable ***search***.

In [24]:
# set grid nilai-nilai pada parameter dengan dictionary data structure
grid = dict()
grid={'max_iter':(1, 10, 100, 1000, 10000),'eta0':(0.0001, 0.001, 0.01, 0.1, 1.0)}
# Jalankan training melalui proses cross validation setting cv
# dan melalui kombinasi 5 nilai pada max_iter dan eta0 tersebut
search = GridSearchCV(model, grid, scoring='accuracy', cv=cv, n_jobs=-1)

Melalui variable ***search***, train model klasifikasi perceptron ini menggunakan data buatan yang telah di jana. Hasil dapatan dari proses training ini akan di simpan di dalam variable ***results***.

In [25]:
# train model dengan cross validation, beserta kombinasi pelbagai 
# nilai learning rate dan iteration dan pulangkan hasil dapatan
# di variable results
results = search.fit(X, y)

Melalui variable results tadi, dengan mudahnya kita boleh mempamirkan maklumat,

*   Purata prestasi ketepatan ramalan terbaik
*   Kombinasi learning rate & iteration terbaik
*   Senarai lengkap purata prestasi ketepatan ramalan dengan setiap kombinasi learning & iteration. 

Merujuk pada maklumat yang dipaparkan di bawah, ketepatan tertinggi adalah **85.7%** manakala kombinasi parameter learning rate & iteration terbaik yang dicadangkan adalah **eta0=0.0001** (lagi besar learning rate lagi cepat proses training selesai) &  **max_iter=10** (lagi kurang iteration, lagi cepat proses training selesai).

In [26]:
# summarize
print('Mean Accuracy: %.3f' % results.best_score_)
print('Config: %s' % results.best_params_)
# summarize all
means = results.cv_results_['mean_test_score']
params = results.cv_results_['params']
for mean, param in zip(means, params):
    print(">%.3f with: %r" % (mean, param))

Mean Accuracy: 0.857
Config: {'eta0': 0.0001, 'max_iter': 10}
>0.850 with: {'eta0': 0.0001, 'max_iter': 1}
>0.857 with: {'eta0': 0.0001, 'max_iter': 10}
>0.857 with: {'eta0': 0.0001, 'max_iter': 100}
>0.857 with: {'eta0': 0.0001, 'max_iter': 1000}
>0.857 with: {'eta0': 0.0001, 'max_iter': 10000}
>0.850 with: {'eta0': 0.001, 'max_iter': 1}
>0.857 with: {'eta0': 0.001, 'max_iter': 10}
>0.857 with: {'eta0': 0.001, 'max_iter': 100}
>0.857 with: {'eta0': 0.001, 'max_iter': 1000}
>0.857 with: {'eta0': 0.001, 'max_iter': 10000}
>0.850 with: {'eta0': 0.01, 'max_iter': 1}
>0.846 with: {'eta0': 0.01, 'max_iter': 10}
>0.853 with: {'eta0': 0.01, 'max_iter': 100}
>0.853 with: {'eta0': 0.01, 'max_iter': 1000}
>0.853 with: {'eta0': 0.01, 'max_iter': 10000}
>0.850 with: {'eta0': 0.1, 'max_iter': 1}
>0.836 with: {'eta0': 0.1, 'max_iter': 10}
>0.847 with: {'eta0': 0.1, 'max_iter': 100}
>0.847 with: {'eta0': 0.1, 'max_iter': 1000}
>0.847 with: {'eta0': 0.1, 'max_iter': 10000}
>0.850 with: {'eta0': 1.0, '

Ok. Mari kita cuba train model ini semula dengan kombinasi learning rate & iteration yang terbaik, kemudian mengujinya dengan test data. Bagaimana? boleh dapat prestasi yang lebih kurang macam masa training tadi?

Cuba main-main dengan setting parameter. Tukar nilai parameter pada fungsi make_classification (n_samples, n_feature, n_informative, n_redundant, dan random_state) dan juga pada fungsi-fungsi lain seperti pada parameter test_size, max_iter, eta0. Kemudian buat kesimpulan hasil daripada penyelidikan anda ini.

Oh ya. Jika anda nak cuba menjalankan kod-kod ini, klik sahaja butang ikon "**Open in Colab**" yang berada bahagian atas artikel ini.

In [50]:
# import library sklearn
from sklearn.datasets import make_classification
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split

# Jana data buatan untuk training data & testing data (X->Input y->label). 
# Pastikan guna nilai random_state yang sama dengan sebelum ini. 
X, y = make_classification(n_samples=1250, n_features=10, n_informative=10, n_redundant=0, random_state=1)

# Asingkan training data & testing data 
# (testing data = 0.2 x 1250 total data => Train Data = 1000, Test Data = 250)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Bina architecture perceptron algoritma dengan setting terbaik 
# iteration & learning rate
model = Perceptron(max_iter=10,eta0=0.0001)

# Train model dengan training data
results = model.fit(X_train, y_train)

# Paparkan prestasi ketepatan ramalan dengan testing data
results.score(X_test,y_test)



0.804

Ok lah. Sampai di sini sahaja dahulu. InsyaAllah kita akan bertemu kembali. ᕕ( ᐛ )ᕗ